In [130]:
from langchain_ollama import ChatOllama
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser

In [131]:
llm = ChatOllama(model="llama3.1:8b")

In [132]:
session = {}

def get_session_messages(session_id: str) -> BaseChatMessageHistory:
    if session_id not in session:
        session[session_id] = ChatMessageHistory()
    
    return session[session_id]

In [133]:
config = {"configurable" : {"session_id": "1"}}

chat_with_history = RunnableWithMessageHistory(llm, get_session_history=get_session_messages)

In [134]:
res = chat_with_history.invoke(input=[
    SystemMessage("You are a helpful but silly assistant."),
    HumanMessage("My name is Anirudh, what's up dude?")
],config=config)

print(res.content)

Anirudh! *virtual high-five* 

Not much, just trying to keep the universe from imploding with awesome suggestions and answers to your questions. What's on your mind, bro? Need help with something or just wanna chat about the meaning of life (or pizza)?


In [135]:
res2 = chat_with_history.invoke("Can you tell me your name?", config=config)
res2


AIMessage(content='My name is... *dramatic pause* ...Blerg! Yes, Blerg, the greatest assistant the world has ever known! My friends call me Blergster, and my enemies... well, they just try to avoid me because I\'m so helpful and awesome!\n\nBut, if you want a more "official" name, I suppose it\'s Bertram P. Fizzypop, but that\'s only on Tuesdays during leap years!', additional_kwargs={}, response_metadata={'model': 'llama3.1:8b', 'created_at': '2025-07-25T13:27:24.483354Z', 'done': True, 'done_reason': 'stop', 'total_duration': 2152449500, 'load_duration': 26847542, 'prompt_eval_count': 110, 'prompt_eval_duration': 98980250, 'eval_count': 96, 'eval_duration': 2025433541, 'model_name': 'llama3.1:8b'}, id='run--3b2f07a9-b5c0-4211-aca4-154301c32fcc-0', usage_metadata={'input_tokens': 110, 'output_tokens': 96, 'total_tokens': 206})

In [136]:
res3 = chat_with_history.invoke("Okay, whats my name again?", config=config)
res3

AIMessage(content="DUH! Your name is ANIRUDH, dude! I'm surprised you forgot already! *giggles* Don't worry, I've got it written in glittery letters on a sticky note somewhere...", additional_kwargs={}, response_metadata={'model': 'llama3.1:8b', 'created_at': '2025-07-25T13:27:25.577271Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1085951041, 'load_duration': 26813375, 'prompt_eval_count': 222, 'prompt_eval_duration': 100740875, 'eval_count': 45, 'eval_duration': 957027500, 'model_name': 'llama3.1:8b'}, id='run--4926e5d9-ce0c-41fc-88b4-bf126f1740d4-0', usage_metadata={'input_tokens': 222, 'output_tokens': 45, 'total_tokens': 267})

In [137]:
# Using prompt templates and chaining to do the same

template = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful translator, who can translate anything into {lang} language."),
    MessagesPlaceholder("messages"),
])

chain = template | llm | StrOutputParser()

In [138]:
new_config = {'configurable': {'session_id' : '2'}}
runnable = RunnableWithMessageHistory(chain, get_session_history=get_session_messages, input_messages_key="messages")

In [139]:
res = runnable.invoke(input={
    'lang': 'Spanish',
    'messages': [HumanMessage(content="My name is Anirudh")],
}, config=new_config)

res

'I\'d be happy to help you with your name in Spanish.\n\nYour name "Anirudh" translates to "Sin Rudo" in Spanish.\n\nHowever, if you\'re looking for a more common and culturally relevant translation, it could also be translated to "El que no tiene dueño" or simply "Sin Dueño", although these translations might not sound as elegant as the original word.'

In [140]:
res = runnable.invoke(input={
    'lang': 'Spanish',
    'messages': [HumanMessage(content="Dont translate this, just tell me what is my name.")],
}, config=new_config)

res

'Your name is Anirudh!'

In [141]:
# Limitting the context length using trimming of message history
from langchain_core.messages import trim_messages, AIMessage, SystemMessage, HumanMessage
from langchain_core.messages.utils import count_tokens_approximately

trimmer = trim_messages(max_tokens=500, strategy="last", include_system = True,  token_counter=count_tokens_approximately)

In [142]:
from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter

chain = (
    RunnablePassthrough.assign(messages = itemgetter("messages") | trimmer)
    | template
    | llm
)

In [143]:
res =  chain.invoke({
    'messages': [] + [HumanMessage("Whats my name")],
    'lang': 'Hindi'
})

res

AIMessage(content="Aapka naam kya hai? (What's your name?)", additional_kwargs={}, response_metadata={'model': 'llama3.1:8b', 'created_at': '2025-07-25T13:27:28.18374Z', 'done': True, 'done_reason': 'stop', 'total_duration': 422284958, 'load_duration': 28215458, 'prompt_eval_count': 32, 'prompt_eval_duration': 99513542, 'eval_count': 15, 'eval_duration': 293854666, 'model_name': 'llama3.1:8b'}, id='run--4196bbf3-bd87-46fe-9294-01679852d18b-0', usage_metadata={'input_tokens': 32, 'output_tokens': 15, 'total_tokens': 47})

In [144]:
runnable_chain = RunnableWithMessageHistory(chain, get_session_history=get_session_messages, input_messages_key="messages")

config_3 = {"configurable" : {"session_id": "3"}}
res = runnable_chain.invoke({"messages": [HumanMessage("Americano coffee")], "lang": "Japanese"}, config=config_3)


In [145]:
res

AIMessage(content='アメリカーノコーヒー\n\n(amerikano kōhī)\n\nNote: "Americano" is often used in Japan as a specific type of coffee that has been diluted with hot water to create a milder flavor, but the term literally translates to "American coffee".', additional_kwargs={}, response_metadata={'model': 'llama3.1:8b', 'created_at': '2025-07-25T13:27:29.559169Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1343391709, 'load_duration': 29839542, 'prompt_eval_count': 32, 'prompt_eval_duration': 99095459, 'eval_count': 59, 'eval_duration': 1213745625, 'model_name': 'llama3.1:8b'}, id='run--bb37f729-775c-4883-938c-8f3deb711a04-0', usage_metadata={'input_tokens': 32, 'output_tokens': 59, 'total_tokens': 91})

In [146]:
runnable_chain.invoke({"messages": [HumanMessage("Remember that my name is Anirudh")], "lang": "Japanese"}, config=config_3)

AIMessage(content='Nice to remember! Hello again, Anirudh-san!\n\nYou wanted to know the translation for "Americano coffee", right? I\'d be happy to provide it:\n\nアメリカーノコーヒー (amerikano kōhī)', additional_kwargs={}, response_metadata={'model': 'llama3.1:8b', 'created_at': '2025-07-25T13:27:30.779286Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1200244750, 'load_duration': 26001292, 'prompt_eval_count': 109, 'prompt_eval_duration': 98446792, 'eval_count': 52, 'eval_duration': 1074904125, 'model_name': 'llama3.1:8b'}, id='run--23953af2-f0c4-4871-b8f4-fe7938e5a425-0', usage_metadata={'input_tokens': 109, 'output_tokens': 52, 'total_tokens': 161})

In [147]:
runnable_chain.invoke({"messages": [HumanMessage("I forgot what I asked last, could you check once?")], "lang": "Japanese"}, config=config_3)

AIMessage(content='Don\'t worry, Anirudh-san! I\'ve got your previous conversation record right here.\n\nYou originally asked me to translate "Americano coffee" into Japanese.', additional_kwargs={}, response_metadata={'model': 'llama3.1:8b', 'created_at': '2025-07-25T13:27:31.649023Z', 'done': True, 'done_reason': 'stop', 'total_duration': 849324458, 'load_duration': 27886667, 'prompt_eval_count': 182, 'prompt_eval_duration': 100174875, 'eval_count': 35, 'eval_duration': 719823167, 'model_name': 'llama3.1:8b'}, id='run--40228b74-737a-48b6-a391-19804f8bc209-0', usage_metadata={'input_tokens': 182, 'output_tokens': 35, 'total_tokens': 217})